In [1]:
from pypendula import PyPendula
from matplotlib import pyplot as plt
import numpy as np

In [2]:
FPS = 120
ALPHA = 3
BETA = 2048

In [3]:
p = PyPendula(fps=120, ics=np.array([np.pi / 5, np.pi / 4, np.pi / 3, 0, 0, 0]))
# p.solve_symbolic()
# p.solve_numeric()
p.simulate()

Solving Symbolic Problem... Done!
Caching Solution... Done!
Solving Numeric Problem... Done!
Setting Up Simulation... Done!


MovieWriter ffmpeg unavailable; using Pillow instead.


: 

In [ ]:
from sympy.physics.mechanics import *
from sympy import symbols, sin, cos, atan, Matrix, simplify, solve
from sympy.utilities import lambdify
from functools import partial
import numpy as np

rng = np.random.default_rng()

In [ ]:
N = 2

q = dynamicsymbols(f'q:{N}')
p = dynamicsymbols(f'p:{N}')
dq = dynamicsymbols(f'q:{N}', level=1)
dp = dynamicsymbols(f'p:{N}', level=1)
m = symbols(f'm:{N}')
l = symbols(f'l:{N}')
c, g, t = symbols('c g t')

# Compose World Frame
O = ReferenceFrame('O')
pin = Point('O*')
pin.set_vel(O, 0)

# Initial Mass
points = [pin.locatenew(f'P_0', l[0] * sin(q[0]) * O.x + l[0] * cos(q[0]) * O.y),]
points[0].set_vel(O, points[0].pos_from(pin).dt(O))
particles = [Particle(f'pP_0', points[0], m[0]),]
particles[0].potential_energy = -m[0] * g * points[0].pos_from(pin).dot(O.y)

for i in range(1, N):
    points.append(pin.locatenew(f'P_{i}', 
        points[i - 1].pos_from(pin) + l[i] * sin(q[i]) * O.x + l[i] * cos(q[i]) * O.y)
    )
    points[i].set_vel(O, points[i].pos_from(pin).dt(O))
    particles.append(Particle(f'pP_{i}', points[i], m[i]))
    particles[i].potential_energy = m[i] * g * points[i].pos_from(pin).dot(O.y)

L = Lagrangian(O, *particles)
LM = LagrangesMethod(L, q, frame=O)  # , forcelist=[(P, -c * P.vel(O))])

In [ ]:
EL = simplify(LM.form_lagranges_equations().subs([(dq[_], p[_]) for _ in range(N)]))
EL

In [ ]:
EOM = simplify(LM.eom.subs([(dq[_], p[_]) for _ in range(N)]))
EOM

In [ ]:
solved_EOM = Matrix(list(solve(EOM, *dp).values()))
solved_EOM